<a href="https://colab.research.google.com/github/NugrahaC/Data-Analyst-Training/blob/main/STOCKMARKET/stock_market_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install yfinance


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-09-25  174.199997  176.970001  174.149994  176.080002  175.848328   
       2023-09-26  174.820007  175.199997  171.660004  171.960007  171.733749   
       2023-09-27  172.619995  173.039993  169.050003  170.429993  170.205750   
       2023-09-28  169.339996  172.029999  167.619995  170.690002  170.465424   
       2023-09-29  172.020004  173.070007  170.339996  171.210007  170.984741   

                     Volume  
Ticker Date                  
AAPL   2023-09-25  46172700  
       2023-09-26  64588900  
       2023-09-27  66921800  
      

In [3]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-09-25  174.199997  176.970001  174.149994  176.080002   
1   AAPL 2023-09-26  174.820007  175.199997  171.660004  171.960007   
2   AAPL 2023-09-27  172.619995  173.039993  169.050003  170.429993   
3   AAPL 2023-09-28  169.339996  172.029999  167.619995  170.690002   
4   AAPL 2023-09-29  172.020004  173.070007  170.339996  171.210007   

    Adj Close    Volume  
0  175.848328  46172700  
1  171.733749  64588900  
2  170.205750  66921800  
3  170.465424  56294400  
4  170.984741  51814200  


In [4]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [5]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [6]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
59  195.314003  192.834500
60  195.666003  193.109000
61  195.917003  193.318501
62  195.958002  193.487000
63  195.747002  193.668501

[64 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
192         NaN         NaN
193         NaN         NaN
194         NaN         NaN
195         NaN         NaN
196         NaN         NaN
..          ...         ...
251  134.544998  135.558499
252  135.115999  135.567500
253  135.939000  135.619500
254  136.274001  135.708500
255  136.882001  135.933500

[64 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
..          ...         ...
123  371.585999  374.21

In [7]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [ ]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [8]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()